## Preprocessing

In [4]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA


# Import pandas and read the charity_data.csv from the provided cloud URL.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [5]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=["EIN", "NAME"])

In [6]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [7]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
application_counts = application_df['APPLICATION_TYPE'].value_counts()
application_counts

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [8]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(application_counts[application_counts < 500].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [9]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
classification_counts = application_df['CLASSIFICATION'].value_counts()
classification_counts

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [10]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts[classification_counts > 1]

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64

In [11]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(classification_counts[classification_counts < 100].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      669
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Name: count, dtype: int64

In [12]:
# Convert categorical data to numeric with `pd.get_dummies`
# Include all object type or categorical columns that may have been missed initially
categorical_cols = application_df.select_dtypes(include=['object']).columns.tolist()
application_df = pd.get_dummies(application_df, columns=categorical_cols, drop_first=True, dtype=float)
application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,ORGANIZATION_Trust,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,108590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,6692,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,142590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL"].values
X = application_df.drop(columns=["IS_SUCCESSFUL"]).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [15]:
# Apply Feature Selection (Remove Least Important Features)
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

# Use the column names from the original DataFrame
feature_importance = pd.DataFrame({'Feature': application_df.drop(columns=["IS_SUCCESSFUL"]).columns, 'Importance': rf.feature_importances_})
feature_importance.sort_values(by='Importance', ascending=False, inplace=True)
low_importance_features = feature_importance[feature_importance['Importance'] < 0.01]['Feature'].tolist()

# Drop low-importance features from the original DataFrame
X_train = pd.DataFrame(X_train, columns=application_df.drop(columns=["IS_SUCCESSFUL"]).columns).drop(columns=low_importance_features).values
X_test = pd.DataFrame(X_test, columns=application_df.drop(columns=["IS_SUCCESSFUL"]).columns).drop(columns=low_importance_features).values

# Apply PCA for Dimensionality Reduction (Fixed to avoid errors)
n_components = min(X_train.shape[1], 15)  # Ensures PCA does not exceed available features
pca = PCA(n_components=n_components)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# Scale Data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_pca)
X_test_scaled = scaler.transform(X_test_pca)


## Compile, Train and Evaluate the Model

In [16]:
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(Dense(256, activation='swish', input_dim=X_train_scaled.shape[1]))
nn.add(BatchNormalization())
nn.add(Dropout(0.2))

# Second hidden layer
nn.add(Dense(128, activation='swish'))
nn.add(BatchNormalization())
nn.add(Dropout(0.2))

# Third hidden layer
nn.add(Dense(64, activation='swish'))
nn.add(BatchNormalization())

# Output layer
nn.add(Dense(1, activation='sigmoid'))

# Check model structure
nn.summary()

c:\Users\olive\anaconda3\envs\dev\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_8 (Dense)                 │ (None, 256)            │         4,096 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_8           │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 47,105 (184.00 KB)

 Trainable params: 46,209 (180.50 KB)

 Non-trainable params: 896 (3.50 KB)

In [17]:
# Compile the model
nn.compile(optimizer=Adam(learning_rate=0.0005), loss='binary_crossentropy', metrics=['accuracy'])

# Create a ModelCheckpoint callback to save weights (adjust frequency as needed) ###
from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint("weights_epoch_{epoch:02d}.h5", save_freq='epoch', verbose=1)

# Implement Callbacks for Optimization
lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)
early_stopper = EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True, verbose=1)
checkpoint = ModelCheckpoint("optimized_weights.h5", monitor='val_loss', save_best_only=True, verbose=1)

In [18]:
# Train the Neural Network with More Epochs
history = nn.fit(
    X_train_scaled, y_train,
    epochs=100,  # Increased epochs for better convergence
    batch_size=16,  # Adjusted batch size for better gradient updates
    validation_data=(X_test_scaled, y_test),
    callbacks=[lr_reducer, early_stopper, checkpoint]
)

Epoch 1/100
1601/1608 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6756 - loss: 0.6449
Epoch 1: val_loss improved from inf to 0.57488, saving model to optimized_weights.h5


1608/1608 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.6757 - loss: 0.6448 - val_accuracy: 0.7170 - val_loss: 0.5749 - learning_rate: 5.0000e-04
Epoch 2/100
1601/1608 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7072 - loss: 0.5912
Epoch 2: val_loss improved from 0.57488 to 0.57261, saving model to optimized_weights.h5


1608/1608 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.7072 - loss: 0.5912 - val_accuracy: 0.7190 - val_loss: 0.5726 - learning_rate: 5.0000e-04
Epoch 3/100
1590/1608 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7107 - loss: 0.5861
Epoch 3: val_loss improved from 0.57261 to 0.56905, saving model to optimized_weights.h5


1608/1608 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.7107 - loss: 0.5861 - val_accuracy: 0.7199 - val_loss: 0.5690 - learning_rate: 5.0000e-04
Epoch 4/100
1598/1608 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7198 - loss: 0.5764
Epoch 4: val_loss improved from 0.56905 to 0.56819, saving model to optimized_weights.h5


1608/1608 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.7198 - loss: 0.5765 - val_accuracy: 0.7214 - val_loss: 0.5682 - learning_rate: 5.0000e-04
Epoch 5/100
1600/1608 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7168 - loss: 0.5816
Epoch 5: val_loss did not improve from 0.56819
1608/1608 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.7168 - loss: 0.5815 - val_accuracy: 0.7201 - val_loss: 0.5717 - learning_rate: 5.0000e-04
Epoch 6/100
1588/1608 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7171 - loss: 0.5791
Epoch 6: val_loss improved from 0.56819 to 0.56806, saving model to optimized_weights.h5


...

1608/1608 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.7246 - loss: 0.5707 - val_accuracy: 0.7237 - val_loss: 0.5604 - learning_rate: 1.2500e-04
Epoch 32/100
1591/1608 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7247 - loss: 0.5657
Epoch 32: val_loss did not improve from 0.56036
1608/1608 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.7247 - loss: 0.5657 - val_accuracy: 0.7230 - val_loss: 0.5611 - learning_rate: 1.2500e-04
Epoch 33/100
1586/1608 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7264 - loss: 0.5709
Epoch 33: val_loss did not improve from 0.56036
1608/1608 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.7265 - loss: 0.5709 - val_accuracy: 0.7241 - val_loss: 0.5609 - learning_rate: 1.2500e-04
Epoch 34/100
1596/1608 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7307 - loss: 0.5652
Epoch 34: ReduceLROnPlateau reducing learning rate to 6.25000029685907e-05.

Epoch 34: val_loss did not improve from 0.56036
1608/1608 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.7307 - loss

1608/1608 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.7261 - loss: 0.5662 - val_accuracy: 0.7247 - val_loss: 0.5600 - learning_rate: 3.1250e-05
Epoch 40/100
1608/1608 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7288 - loss: 0.5655
Epoch 40: val_loss did not improve from 0.56001
1608/1608 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.7288 - loss: 0.5655 - val_accuracy: 0.7249 - val_loss: 0.5603 - learning_rate: 3.1250e-05
Epoch 41/100
1599/1608 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7319 - loss: 0.5627
Epoch 41: val_loss improved from 0.56001 to 0.55978, saving model to optimized_weights.h5


1608/1608 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.7319 - loss: 0.5627 - val_accuracy: 0.7245 - val_loss: 0.5598 - learning_rate: 3.1250e-05
Epoch 42/100
1598/1608 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7280 - loss: 0.5647
Epoch 42: val_loss did not improve from 0.55978
1608/1608 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.7280 - loss: 0.5647 - val_accuracy: 0.7250 - val_loss: 0.5602 - learning_rate: 3.1250e-05
Epoch 43/100
1601/1608 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7233 - loss: 0.5683
Epoch 43: val_loss did not improve from 0.55978
1608/1608 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.7234 - loss: 0.5683 - val_accuracy: 0.7250 - val_loss: 0.5601 - learning_rate: 3.1250e-05
Epoch 44/100
1593/1608 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7270 - loss: 0.5670
Epoch 44: ReduceLROnPlateau reducing learning rate to 1.5625000742147677e-05.

Epoch 44: val_loss did not improve from 0.55978
1608/1608 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.7270 - lo

1608/1608 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.7276 - loss: 0.5669 - val_accuracy: 0.7251 - val_loss: 0.5596 - learning_rate: 1.5625e-05
Epoch 47/100
1596/1608 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7293 - loss: 0.5634
Epoch 47: val_loss did not improve from 0.55965
1608/1608 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7293 - loss: 0.5634 - val_accuracy: 0.7247 - val_loss: 0.5600 - learning_rate: 1.5625e-05
Epoch 48/100
1600/1608 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7321 - loss: 0.5642
Epoch 48: val_loss did not improve from 0.55965
1608/1608 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.7321 - loss: 0.5642 - val_accuracy: 0.7250 - val_loss: 0.5604 - learning_rate: 1.5625e-05
Epoch 49/100
1608/1608 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7243 - loss: 0.5669
Epoch 49: val_loss improved from 0.55965 to 0.55951, saving model to optimized_weights.h5


1608/1608 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.7243 - loss: 0.5669 - val_accuracy: 0.7249 - val_loss: 0.5595 - learning_rate: 1.5625e-05
Epoch 50/100
1603/1608 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7248 - loss: 0.5697
Epoch 50: val_loss did not improve from 0.55951
1608/1608 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.7248 - loss: 0.5696 - val_accuracy: 0.7251 - val_loss: 0.5600 - learning_rate: 1.5625e-05
Epoch 51/100
1595/1608 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7299 - loss: 0.5652
Epoch 51: val_loss did not improve from 0.55951
1608/1608 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.7299 - loss: 0.5653 - val_accuracy: 0.7245 - val_loss: 0.5597 - learning_rate: 1.5625e-05
Epoch 52/100
1604/1608 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7236 - loss: 0.5694
Epoch 52: ReduceLROnPlateau reducing learning rate to 7.812500371073838e-06.

Epoch 52: val_loss did not improve from 0.55951
1608/1608 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.7236 - los

In [19]:
# Evaluate Optimized Model
final_loss, final_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Optimized Model -> Loss: {final_loss}, Accuracy: {final_accuracy}")

268/268 - 0s - 1ms/step - accuracy: 0.7249 - loss: 0.5595
Optimized Model -> Loss: 0.5595135688781738, Accuracy: 0.7248979806900024


In [20]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimized.h5")